In [2]:
! pip install tab_transformer_pytorch gdown -q


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tab_transformer_pytorch import TabTransformer
import torch
import torch.nn as nn
import time

In [4]:
# Display settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None) # Adjust display width

!gdown https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM
df = pd.read_csv('/kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv')
df

Downloading...
From (original): https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM
From (redirected): https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM&confirm=t&uuid=da2a4f02-f418-458e-851b-352806146af0
To: /kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv
100%|█████████████████████████████████████████| 420M/420M [00:02<00:00, 209MB/s]


/tmp/ipykernel_13/3050658293.py:6: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv')


,Unnamed: 0,...1,Hour,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_ID,Incomplete_qSOFA_Score,SOFA_Score,SIRS,NEWS_Score,Apache_II_Score,BUN_Creatinine_Ratio,Shock_Index,Bilirubin_Creatinine_Ratio,datetime
0,0,0,0,65.0,100.0,35.78,129.0,72.0,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,NaN,NaN,-0.02,1,0,17072,0,2,0,0,16,28.750000,0.503876,0.875000,2019-01-01 01:00:00
1,1,1,1,65.0,100.0,35.78,129.0,72.0,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,NaN,NaN,-0.02,2,0,17072,0,2,0,0,16,28.750000,0.503876,0.875000,2019-01-01 02:00:00
2,2,2,2,78.0,100.0,35.78,129.0,42.5,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,NaN,NaN,-0.02,3,0,17072,0,5,0,0,18,28.750000,0.604651,0.875000,2019-01-01 03:00:00
3,3,3,3,73.0,100.0,35.78,129.0,42.5,69.0,17.0,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,NaN,NaN,-0.02,4,0,17072,0,5,0,0,18,28.750000,0.565891,0.875000,2019-01-01 04:00:00
4,4,4,4,70.0,100.0,35.78,129.0,74.0,69.0,14.0,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,NaN,NaN,-0.02,5,0,17072,0,2,0,0,14,28.750000,0.542636,0.875000,2019-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,1552205,21,21,83.0,99.0,36.10,121.0,77.0,54.0,22.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,112.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,1.0,0.0,-2.93,22,0,113911,1,2,0,1,10,45.945946,0.685950,1.891892,NaN
1552206,1552206,22,22,80.0,92.0,36.10,102.0,73.0,51.0,24.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,112.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,1.0,0.0,-2.93,23,0,113911,1,2,0,1,10,45.945946,0.784314,1.891892,NaN
1552207,1552207,23,23,95.0,97.0,36.70,128.5,83.0,58.5,25.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,163.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,1.0,0.0,-2.93,24,0,113911,1,2,1,4,11,45.945946,0.739300,1.891892,NaN
1552208,1552208,24,24,104.0,99.0,36.70,127.0,85.0,59.0,24.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,163.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,1.0,0.0,-2.93,25,0,113911,1,2,1,2,10,45.945946,0.818898,1.891892,NaN


In [5]:
df.columns

Index(['Unnamed: 0', '...1', 'Hour', 'HR', 'O2Sat', 'Temp', 'SBP', 'MAP',
       'DBP', 'Resp', 'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2',
       'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride',
       'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium',
       'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb',
       'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1',
       'Unit2', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_ID',
       'Incomplete_qSOFA_Score', 'SOFA_Score', 'SIRS', 'NEWS_Score',
       'Apache_II_Score', 'BUN_Creatinine_Ratio', 'Shock_Index',
       'Bilirubin_Creatinine_Ratio', 'datetime'],
      dtype='object')

In [6]:

# Dropping unnecessary columns
df.drop(['Unnamed: 0','...1','Hour', 'Unit1', 'Unit2','Apache_II_Score','Patient_ID', 'SOFA_Score', 'SIRS', 'Bilirubin_Creatinine_Ratio'], axis=1, inplace=True)


In [7]:
# Data preparation
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['weekday'] = df['datetime'].dt.weekday
df.drop('datetime', axis=1, inplace=True)

# Fill NaN values
df.fillna(-1, inplace=True)

In [8]:
df.columns

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'HospAdmTime', 'ICULOS',
       'SepsisLabel', 'Incomplete_qSOFA_Score', 'NEWS_Score',
       'BUN_Creatinine_Ratio', 'Shock_Index', 'hour', 'day', 'weekday'],
      dtype='object')

In [9]:
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,Incomplete_qSOFA_Score,NEWS_Score,BUN_Creatinine_Ratio,Shock_Index,hour,day,weekday
0,65.0,100.0,35.78,129.0,72.0,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,-0.02,1,0,0,0,28.750000,0.503876,1.0,1.0,1.0
1,65.0,100.0,35.78,129.0,72.0,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,-0.02,2,0,0,0,28.750000,0.503876,2.0,1.0,1.0
2,78.0,100.0,35.78,129.0,42.5,69.0,16.5,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,-0.02,3,0,0,0,28.750000,0.604651,3.0,1.0,1.0
3,73.0,100.0,35.78,129.0,42.5,69.0,17.0,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,-0.02,4,0,0,0,28.750000,0.565891,4.0,1.0,1.0
4,70.0,100.0,35.78,129.0,74.0,69.0,14.0,34.0,6.0,26.0,0.40,7.61,27.0,98.0,29.0,23.0,152.0,9.6,104.0,0.80,2.6,161.0,1.90,1.6,2.1,3.2,0.7,0.50,29.7,9.5,30.6,11.3,716.0,330.0,68.54,0,-0.02,5,0,0,0,28.750000,0.542636,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,83.0,99.0,36.10,121.0,77.0,54.0,22.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,112.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,-2.93,22,0,1,1,45.945946,0.685950,-1.0,-1.0,-1.0
1552206,80.0,92.0,36.10,102.0,73.0,51.0,24.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,112.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,-2.93,23,0,1,1,45.945946,0.784314,-1.0,-1.0,-1.0
1552207,95.0,97.0,36.70,128.5,83.0,58.5,25.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,163.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,-2.93,24,0,1,4,45.945946,0.739300,-1.0,-1.0,-1.0
1552208,104.0,99.0,36.70,127.0,85.0,59.0,24.0,42.0,3.3,28.5,0.32,7.36,50.0,95.6,27.0,17.0,38.0,9.2,98.0,0.37,0.2,163.0,1.16,2.3,3.5,4.5,0.7,0.01,33.8,10.3,29.7,5.7,366.0,260.0,88.00,0,-2.93,25,0,1,2,45.945946,0.818898,-1.0,-1.0,-1.0


In [10]:

# Addressing class imbalance
class_labels = np.unique(df['SepsisLabel'])
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=df['SepsisLabel'].values)
class_weight_dict = {class_label: weight for class_label, weight in zip(class_labels, class_weights)}
print("Class Weight Dictionary:", class_weight_dict)


Class Weight Dictionary: {0: 0.5091570261380023, 1: 27.801440034388882}


In [11]:
# Split the data into features (X) and target (y)
X = df.drop(['SepsisLabel'], axis=1)
y = df['SepsisLabel'].values

In [15]:

# Separate categorical and continuous features
categorical_features = ['Gender', 'Incomplete_qSOFA_Score', 
        'NEWS_Score','hour', 'day', 
       'weekday']
continuous_features = [col for col in X.columns if col not in categorical_features]

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    X[feature] = label_encoders[feature].fit_transform(X[feature])

In [16]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize continuous features
scaler = StandardScaler()
X_train[continuous_features] = scaler.fit_transform(X_train[continuous_features])
X_test[continuous_features] = scaler.transform(X_test[continuous_features])


In [17]:

# Convert data to PyTorch tensors
X_train_categ = torch.tensor(X_train[categorical_features].values, dtype=torch.long)
X_train_cont = torch.tensor(X_train[continuous_features].values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

In [18]:

X_test_categ = torch.tensor(X_test[categorical_features].values, dtype=torch.long)
X_test_cont = torch.tensor(X_test[continuous_features].values, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [19]:

# Initialize the Tab Transformer model
model = TabTransformer(
    categories=[len(label_encoders[feat].classes_) for feat in categorical_features],
    num_continuous=len(continuous_features),
    dim=32,
    dim_out=1,
    depth=6,
    heads=8,
    attn_dropout=0.1,
    ff_dropout=0.1,
    mlp_hidden_mults=(4, 2),
    mlp_act=nn.ReLU(),
    continuous_mean_std=None
)


In [20]:
import torch.nn as nn

# Initialize early stopping attributes
best_loss = np.inf
patience = 10
patience_counter = 0

In [21]:
# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(class_weights[1] / class_weights[0]))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Training loop
num_epochs = 250
batch_size = 2500 # Batch size
patience = 10
best_val_auc = 0
counter = 0
start_time = time.time()

try:
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        
        # Create batches
        perm = torch.randperm(len(X_train_categ))
        for i in range(0, len(X_train_categ), batch_size):
            batch_idx = perm[i:i+batch_size]
            
            # Forward pass
            outputs = model(X_train_categ[batch_idx], X_train_cont[batch_idx])
            loss = criterion(outputs, y_train[batch_idx])
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= (len(X_train_categ) // batch_size)
        
        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_categ, X_test_cont)
            val_loss = criterion(outputs, y_test).item()
            
            prob_predictions = torch.sigmoid(outputs).squeeze().numpy()
            val_auc = roc_auc_score(y_test, prob_predictions)
        
        epoch_time = time.time() - start_time
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}, Time: {epoch_time:.2f}s")
        
        # Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break
        
        start_time = time.time()
        
except KeyboardInterrupt:
    print("Training interrupted.")

Epoch [1/250], Train Loss: 1.1899, Val Loss: 1.1581, Val AUC: 0.7531, Time: 76.61s
Epoch [2/250], Train Loss: 1.1393, Val Loss: 1.1217, Val AUC: 0.7746, Time: 76.97s
Epoch [3/250], Train Loss: 1.1039, Val Loss: 1.0877, Val AUC: 0.7937, Time: 76.76s
Epoch [4/250], Train Loss: 1.0665, Val Loss: 1.0581, Val AUC: 0.8062, Time: 76.67s
Epoch [5/250], Train Loss: 1.0317, Val Loss: 1.0284, Val AUC: 0.8181, Time: 77.31s
Epoch [6/250], Train Loss: 1.0014, Val Loss: 1.0084, Val AUC: 0.8280, Time: 76.42s
Epoch [7/250], Train Loss: 0.9725, Val Loss: 0.9801, Val AUC: 0.8374, Time: 77.38s
Epoch [8/250], Train Loss: 0.9424, Val Loss: 0.9685, Val AUC: 0.8461, Time: 76.83s
Epoch [9/250], Train Loss: 0.9136, Val Loss: 0.9387, Val AUC: 0.8548, Time: 76.72s
Epoch [10/250], Train Loss: 0.8828, Val Loss: 0.9179, Val AUC: 0.8598, Time: 76.68s
Epoch [11/250], Train Loss: 0.8505, Val Loss: 0.8925, Val AUC: 0.8708, Time: 76.39s
Epoch [12/250], Train Loss: 0.8197, Val Loss: 0.8666, Val AUC: 0.8782, Time: 76.58s
E

In [23]:
torch.save(model.state_dict(), '1-Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.pth')


In [24]:
# Prediction on the test set
predictions = (torch.sigmoid(model(X_test_categ, X_test_cont)) > 0.5).float().squeeze().numpy()
prob_predictions = torch.sigmoid(model(X_test_categ, X_test_cont)).detach().squeeze().numpy()

# Evaluation metrics
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)
auprc = average_precision_score(y_test, prob_predictions)
classification_report_output = classification_report(y_test, predictions)
confusion_matrix_report = confusion_matrix(y_test, predictions)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("AUPRC:", auprc)
print("Classification Report:")
print(classification_report_output)
print("Confusion Matrix:")
print(confusion_matrix_report)

Accuracy: 0.9768459164674883
F1 Score: 0.5529850746268656
ROC AUC: 0.9724816477248168
AUPRC: 0.6207028050987418
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99    304859
         1.0       0.42      0.80      0.55      5583

    accuracy                           0.98    310442
   macro avg       0.71      0.89      0.77    310442
weighted avg       0.99      0.98      0.98    310442

Confusion Matrix:
[[298808   6051]
 [  1137   4446]]


In [25]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score

# Function to calculate precision, recall, F1, and F2 scores
def calculate_metrics(y_true, y_pred, beta=1):
    # Binarize predictions based on a threshold (0.5)
    y_pred = (y_pred > 0.5).astype(int)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    f2 = fbeta_score(y_true, y_pred, beta=2)  # beta=2 gives twice as much importance to recall as to precision
    
    return accuracy, precision, recall, f1, f2

# Assume model, X_test_categ, X_test_cont, y_test are defined
# Make sure the model is in evaluation mode
model.eval()

# No gradient needed for evaluation
with torch.no_grad():
    # Forward pass through the model to get the raw output
    outputs = model(X_test_categ, X_test_cont)
    
    # Convert outputs to probabilities (assuming binary classification and sigmoid activation)
    prob_predictions = torch.sigmoid(outputs).squeeze().numpy()
    
    # Calculate metrics
    accuracy, precision, recall, f1, f2 = calculate_metrics(y_test.numpy(), prob_predictions)
    
    # Print the evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"F2 Score: {f2:.4f}")


Accuracy: 0.9770
Precision: 0.4256
Recall: 0.8006
F1 Score: 0.5558
F2 Score: 0.6807
